In [16]:
import numpy as np
import pandas as pd
import os
from lr.text_processing.util import pre_process_nli_df  # noqa
from lr.text_processing.util import get_corpus  # noqa
from lr.training.util import get_ternary_label, filter_df_by_label  # noqa
from lr.training.language_representation import Tfidf  # noqa
from lr.models.xgb import XGBCWrapper  # noqa

In [ ]:
## Variables

In [17]:
train_path = "data/toy/train.csv"
dev_path = "data/toy/dev.csv"

search_path = "hyperparams/xgb_snli/search_455.csv"

verbose = True
n_jobs = 3
random_state = 34
max_features = None
dgp_seed = 342

In [ ]:
## Get data

In [18]:
train = pd.read_csv(train_path)
dev = pd.read_csv(dev_path)
train = filter_df_by_label(train.dropna()).reset_index(drop=True)
dev = filter_df_by_label(dev.dropna()).reset_index(drop=True)
pre_process_nli_df(train)
pre_process_nli_df(dev)

In [19]:
## Get hyperarams

In [15]:
params_keys = ['n_estimators', 'max_depth', "reg_alpha",
               "reg_gamma", "learning_rate", "subsample", "colsample_bytree"]

hyperparams = {"RepresentationFunction": Tfidf,
               "random_state": random_state,
               "n_jobs": n_jobs,
               "max_features": max_features,
               "label_translation": get_ternary_label,
               "dgp_seed": dgp_seed,
               "data_set_name": "snli",
               "transformation_name": None,
               "rho": 0.3,
               "model_name_or_path": "gradient boosting",
               "number_of_simulations": 10000,
                "output_dir": None,
               "verbose": verbose}

search_results = pd.read_csv(search_path) 
               
for k in params_keys:
    hyperparams[k] = search_results.loc[0,k]

0.32

In [ ]:
model = XGBCWrapper(hyperparams)
model.fit(train)
model.get_acc(dev)